# Week 3

## Scrape Wiki Page

<strong>-Import packages</strong>

In [2]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as bs

print('Imported all packages')

Imported all packages


### Scrape the web page's HTML

In [3]:
# URL To scrape from
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
# Request the raw HTML of the page at the URL
raw_page = rq.get(URL)

In [5]:
# Convert the raw HTML into a BS Parsable format
soup = bs(raw_page.text)

In [6]:
# Find the Table elements on the page
soup_table = soup.find_all('tbody')
# Looking at only the first table on the page
# Create a list of all the rows and their values
soup_rows = soup_table[0].find_all('tr')
# Use select to create a list of all Table cells
# On the webpage
soup_select = soup.select('td')
# Strip all of the html off of the values
soup_select[2].text.strip()
len(soup_select)

898

### Convert the list of Table Elements into Nx3 List/DF

In [7]:
table = []
length = 863
i = 0
while i < length:
    row = []
    for k in range(3):
        if (i < length):
            row.append(
                soup_select[i].text.strip()
            )
            i += 1

    table.append(row)

In [8]:
df = pd.DataFrame(table)

### Change the Column Headers

In [9]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

### Clean the DF

In [10]:
dft = df

In [11]:
# Replace all 'Not assigned' strings with Numpy.NaN
df = df.replace('Not assigned', np.NaN)
# Fill in the remaining cells that don't have a value
# will a Numpy.NaN
df.fillna(value=pd.np.NaN, inplace=True)
# Drop every row where there is a Np.NaN in the
# Borough column
df.dropna(axis=0, subset=['Borough'], inplace=True)
# Replace all neighborhoods without values with the
# values of the borough that they are in
df['Neighborhood'].fillna(df['Borough'], inplace=True)

### Groupby The Postal Code and Concatinate the Neighborhoods

In [12]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [13]:
df.shape

(103, 3)

# Geo Decoder

In [14]:
import geocoder


def get_coords(postal_code):
    lat_lng_coords = None
    while (lat_lng_coords is None):
        g = geocoder.google('{}, Toranto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return (lat_lng_coords[0], lat_lng_coords[1])
    

In [18]:
coords = pd.read_csv('Geospatial_Coordinates.csv')

In [33]:
l = coords.loc[coords['Postal Code'] == 'M9V'].iloc[0]

In [38]:
df['Latitude'] = coords['Latitude']
df['Longitude'] = coords['Longitude']

In [42]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [48]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [52]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torono are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [53]:

# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [57]:
CLIENT_ID = 'NAUV4LVLNMZ4NWHJGJFLCCBI3B5CHCA1TDV5N2PUJCX3QK1T' # your Foursquare ID
CLIENT_SECRET = 'ENZMBH04Y1FOY4J4L11V4ISV1EFPCRFP4SUQQCJYUTG2UDKG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [63]:
LIMIT = 100
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df.loc[0, 'Latitude'], 
    df.loc[0, 'Longitude'], 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NAUV4LVLNMZ4NWHJGJFLCCBI3B5CHCA1TDV5N2PUJCX3QK1T&client_secret=ENZMBH04Y1FOY4J4L11V4ISV1EFPCRFP4SUQQCJYUTG2UDKG&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=100'

In [64]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca4fac8db04f507fa6eb9fb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs'

In [65]:
def get_catagory_type(row):
    try:
        catagories_list = row['categories']
    except:
        catagories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

In [69]:
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d669cba83865481c948fa53-0,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",4d669cba83865481c948fa53,8130 Sheppard Ave E,CA,Toronto,Canada,Morningside Ave,595,"[8130 Sheppard Ave E (Morningside Ave), Toront...","[{'label': 'display', 'lat': 43.80228301948931...",43.802283,-79.198565,M1B 3W3,ON,Images Salon & Spa,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b914562f964a520d4ae33e3-1,"[{'id': '4bf58dd8d48988d144941735', 'name': 'C...",4b914562f964a520d4ae33e3,875 Milner Ave,CA,Toronto,Canada,Milner and Morningside,912,"[875 Milner Ave (Milner and Morningside), Toro...","[{'label': 'display', 'lat': 43.79855785997625...",43.798558,-79.195777,M1B,ON,Caribbean Wave,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b6718c2f964a5203f3a2be3-2,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",4b6718c2f964a5203f3a2be3,853 Milner Ave,CA,Scarborough,Canada,at Morningside Ave,796,"[853 Milner Ave (at Morningside Ave), Scarboro...","[{'label': 'display', 'lat': 43.80010575211872...",43.800106,-79.198258,M1B 5N6,ON,Harvey's,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-579a91b3498e9bd833afa78a-3,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",579a91b3498e9bd833afa78a,8129 Sheppard Avenue,CA,Scarborough,Canada,NaN,600,"[8129 Sheppard Avenue, Scarborough ON M1B 6A3,...","[{'label': 'display', 'lat': 43.8020084, 'lng'...",43.802008,-79.198080,M1B 6A3,ON,Wendy's,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bb6b9446edc76b0d771311c-4,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",4bb6b9446edc76b0d771311c,NaN,CA,Toronto,Canada,Morningside & Sheppard,387,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.80744841934756...",43.807448,-79.199056,NaN,ON,Wendy's,0,[]
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bcb612d3740b713f0606265-5,"[{'id': '4bf58dd8d48988d121951735', 'name': 'P...",4bcb612d3740b713f0606265,850 Milner Avenue,CA,Scarborough,Canada,NaN,735,"[850 Milner Avenue, Scarborough ON M1B 5N7, Ca...","[{'label': 'display', 'lat': 43.80028467632823...",43.800285,-79.196607,M1B 5N7,ON,Staples Morningside,0,[]
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b16e23bf964a520edbe23e3-6,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",4b16e23bf964a520edbe23e3,8129 Sheppard Ave,CA,Scarborough,Canada,Morningside Ave,605,"[8129 Sheppard Ave (Morningside Ave), Scarboro...","[{'label': 'display', 'lat': 43.801999861382, ...",43.802000,-79.198169,M1B 6A3,ON,Tim Hortons,0,[]
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bd08ceeb221c9b6cbe8d3d0-7,"[{'id': '4bf58dd8d48988d1fb941735', 'name': 'H...",4bd08ceeb221c9b6cbe8d3d0,1275 Morningside,CA,Toronto,Canada,NaN,580,"[1275 Morningside, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.80316130483184...",43.803161,-79.199681,NaN,ON,Lee Valley,0,[]
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ceaa2f0f8653704f906bec4-8,"[{'id': '4bf58dd8d48988d1c8941735', 'name': 'A...",4ceaa2f0f8653704f906bec4,1161 Morningside Avenue,CA,Toronto,Canada,Morningside &Sheppard,743,[1161 Morningside Avenue (Morningside &Sheppar...,"[{'label': 'display', 'lat': 43.80126221955427...",43.801262,-79.199758,NaN,ON,Mr Jerk,0,[]
9,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c706524df6b8cfab244b84d-9,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",4c706524df6b8cfab244b84d,3-1158 Morningside Ave,CA,Toronto,Canada,Sheppard Ave,791,"[3-1158 Morningside Ave (Sheppard Ave), Toront...","[{'label': 'display', 'lat': 43.8

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [76]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 284 unique categories.


In [84]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
toronto_onehot.shape
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood,Long Branch",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park,Lawrence Manor East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff,Cliffside West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [89]:
toronto_grouped.shape

(99, 284)